In [2]:
import numpy as np
import math

In [3]:
def isCircle(dimension, original):
    errorLevel = 0
    y, x, h, w, cy, cx = dimension
    padding = 0
    tmpImg = original[
        y - padding : y + h + padding, x - padding : x + w + padding
    ].copy()
    theight, twidth, _ = tmpImg.shape
    for i in range(0, theight):
        for j in range(0, twidth):
            if tmpImg[i, j][0] > 250:
                tmpImg[i, j] = np.array((255, 255, 255))
            else:
                tmpImg[i, j] = np.array((0, 0, 0))

    xmin = 999999
    ymin = 999999
    xmax = -1
    ymax = -1

    for i in range(0, theight):
        for j in range(0, twidth):
            if tmpImg[i, j][0] == 255:
                if ymin > i:
                    ymin = i
                if ymax < i:
                    ymax = i
                if xmin > j:
                    xmin = j
                if xmax < j:
                    xmax = j

    w = xmax - xmin
    h = ymax - ymin
    cx = xmin + w / 2
    cy = ymin + h / 2
    # print("w: ", w, " h: ", h, " cx: ", cx, " cy: ", cy)

    if w < 6 or h < 6:
        return 999

    if w - h < -2 or w - h > 2:
        return 999

    minDistances = [99999, 99999]
    for i in range(0, theight):
        for j in range(0, twidth):
            if tmpImg[i, j][0] != 255:
                dist = (cy - i) * (cy - i) + (cx - j) * (cx - j)
                if minDistances[0] > dist:
                    minDistances[0] = dist
                    minDistances = sorted(minDistances, reverse=True)

    maxDistances = list(map(lambda x: 0, minDistances))
    for i in range(0, theight):
        for j in range(0, twidth):
            if tmpImg[i, j][0] == 255:
                dist = (cy - i) * (cy - i) + (cx - j) * (cx - j)
                if maxDistances[0] < dist:
                    maxDistances[0] = dist
                    maxDistances = sorted(maxDistances, reverse=False)

    maxDistances = list(map(lambda x: math.sqrt(x), maxDistances))
    minDistances = list(map(lambda x: math.sqrt(x), minDistances))

    errorLevel = sum(maxDistances) / len(maxDistances) - (sum(minDistances)) / len(
        minDistances
    )
    print(maxDistances, minDistances, errorLevel)

    return errorLevel


In [ ]:


def w_h_ofClusters(points1, points2):
    minX = 999999
    minY = 999999
    maxX = 0
    maxY = 0

    for p in points1:
        if minX >= p[1]:
            minX = p[1]
        if minY >= p[0]:
            minY = p[0]
        if maxX <= p[1]:
            maxX = p[1]
        if maxY <= p[0]:
            maxY = p[0]

    for p in points2:
        if minX >= p[1]:
            minX = p[1]
        if minY >= p[0]:
            minY = p[0]
        if maxX <= p[1]:
            maxX = p[1]
        if maxY <= p[0]:
            maxY = p[0]

    w = maxX - minX
    h = maxY - minY

    return [w, h]


def distSquare(p1, p2):
    return (p1[0] - p2[0]) * (p1[0] - p2[0]) + (p1[1] - p2[1]) * (p1[1] - p2[1])


def minDistSquare_ofClusters(points1, points2):
    minDist = 99999
    for point1 in points1:
        for point2 in points2:
            dist = distSquare(point1, point2)
            if dist < minDist:
                minDist = dist

    return minDist


# sectorDimensions = [y, x, h, w, cy, cx]
def clustering(sectorDimensions, spaceBetweenDotsMax: int, diceWidthMax: int):
    clusters = []

    for dimension in sectorDimensions:
        y, x, h, w, cy, cx = dimension
        if w == 0:
            continue
        if h == 0:
            continue
        clusters.append([cy, cx, dimension])

    clusters = [[x] for x in clusters]

    for spaceBetweenDots in range(1, spaceBetweenDotsMax + 1):
        for diceWidth in range(spaceBetweenDotsMax, diceWidthMax + 1):
            clusters = sorted(clusters, key=lambda a: a[0][0], reverse=True)
            clusters = sorted(clusters, key=lambda a: len(a), reverse=True)
            i = 0
            while i != len(clusters):
                points1 = clusters[i]
                j = i + 1

                while j != len(clusters):
                    points2 = clusters[j]
                    minDist = minDistSquare_ofClusters(points1, points2)
                    w, h = w_h_ofClusters(points1, points2)
                    if (
                        minDist < spaceBetweenDots * spaceBetweenDots
                        and w < diceWidth
                        and h < diceWidth
                    ):
                        [(lambda p: clusters[i].append(p))(p) for p in clusters.pop(j)]
                    else:
                        j = j + 1

                i = i + 1

    i = 0
    while i != len(clusters):
        points1 = clusters[i]
        j = i + 1

        while j != len(clusters):
            points2 = clusters[j]
            w, h = w_h_ofClusters(points1, points2)
            if w < diceWidth and h < diceWidth:
                [(lambda p: clusters[i].append(p))(p) for p in clusters.pop(j)]
            else:
                j = j + 1

        i = i + 1

    return clusters


In [ ]:
class Side:
    dimensions = []
    __isValid = 0

    def __init__(self, original):
        self.original = original

    @property
    def isValid(self):
        if self.__isValid == 1:
            return True

        if self.__isValid == -1:
            return False

        check = True
        for dimension in self.dimensions:
            errorLevel = isCircle(dimension, self.original)
            if errorLevel >= 0.9:
                check = False

        if check:
            self.__isValid = 1
        else:
            self.__isValid = -1

        return check

    @property
    def center(self):
        cxs = list(map(lambda x: x[5], self.dimensions))
        cys = list(map(lambda x: x[4], self.dimensions))
        cx = sum(cxs) / len(cxs)
        cy = sum(cys) / len(cys)
        return [cy, cx]

    @property
    def value(self):
        return len(self.dimensions)


In [ ]:
class Dice:
    sides: list[Side]
    isValid: bool = False
    value: int = -1

    def __init__(self):
        self.sides = []

    @property
    def center(self):
        cxs = list(map(lambda side: side.center[1], self.sides))
        cys = list(map(lambda side: side.center[0], self.sides))
        cx = sum(cxs) / len(cxs)
        cy = sum(cys) / len(cys)
        return [cy, cx]
